<a href="https://colab.research.google.com/github/evlko/CS-388/blob/main/NLP_Lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LABORATY WORK 2 (Information Search)
Downloading the Classic Dataset - Aeronautics Textset CRANFIELD

In [1]:
! wget -q http://ir.dcs.gla.ac.uk/resources/test_collections/cran/cran.tar.gz
! tar -xvf cran.tar.gz
! rm cran.tar.gz*

cran.all.1400
cran.qry
cranqrel
cranqrel.readme


Take only the requests themselves (these will be our documents)

In [2]:
! grep -v "^\." cran.qry > just.qry
! head -3 just.qry

what similarity laws must be obeyed when constructing aeroelastic models
of heated high speed aircraft .
what are the structural and aeroelastic problems associated with flight


Combine multiline into one

In [4]:
raw_query_data = [line.strip() for line in open("just.qry", "r").readlines()]
query_data = [""]

for query_part in raw_query_data:
  query_data[-1] += query_part + " "
  if query_part.endswith("."):
    query_data.append("")

query_data[:2]

['what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft . ',
 'what are the structural and aeroelastic problems associated with flight of high speed aircraft . ']

### Queires to our documents

In [18]:
QUERIES = ['viscous interactions']

## Boolean retrieval
Let's represent each document as a "bitmask": a vector the size of a dictionary, in which each position is one if the document has a corresponding term, and zero if there is no term.

In [ ]:
! pip install -q scikit-learn==0.22.2.post1

In [8]:
from  sklearn.feature_extraction.text import CountVectorizer

encoder = CountVectorizer(binary=True)
encoded_data = encoder.fit_transform(query_data)
encoded_queries = encoder.transform(QUERIES)
list(encoder.vocabulary_)[:3]

['what', 'similarity', 'laws']

Let's look at the presentation of the first sentence

In [9]:
id2term = {idx: term for term, idx in encoder.vocabulary_.items()}
non_zero_values_ids = encoded_data[0].nonzero()[1]

terms = [id2term[idx] for idx in non_zero_values_ids]
terms

['what',
 'similarity',
 'laws',
 'must',
 'be',
 'obeyed',
 'when',
 'constructing',
 'aeroelastic',
 'models',
 'of',
 'heated',
 'high',
 'speed',
 'aircraft']

## Task 0

Now for each of these `QUERIES` queries, we will find the closest document for it from `query_data` by Jaccard's similarity. There are better ways to do this, but you need to implement the Jaccard distance and then apply it to our data.

In [12]:
import numpy as np 

def jaccard_sim(vector_a: np.array, vector_b: np.array) -> float:
  """
    Similarity or Jaccard Coefficient: ratio of the intersect power
    to the union power
  """
  return sum(vector_a * vector_b) / (sum(vector_a) + sum(vector_b) - sum(vector_a * vector_b))
  
# Simple text
assert jaccard_sim(np.array([1, 0, 1, 0, 1]), np.array([0, 1, 1, 1, 1])) == 0.4

Here the documents are represented in the same way as the rows in the matrix of document-terms. Each cell of the vector is responsible for the presence / absence of a specific element (for example, a term word, when we have only 5 words in the dictionary). In the first case there are 3, in the second there are 4. The union is all 5 possible elements. Intersection is 2. Hence the ratio is 2/5 or just 0.4.

## Task 1
Compute for each query the closest documents.

In [17]:
for q_id, query in enumerate(encoded_queries):
  # cast to the desired type
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in encoded_data]
  # calc jaccard sim
  id2doc2similarity = [(doc_id, doc, jaccard_sim(query, doc)) for doc_id, doc in enumerate(docs)]
  # sort
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=True)
  
  print("Q: %s\nFOUND:" % QUERIES[q_id])
  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))

Q: viscous interactions
FOUND:
    44	0.15	has anyone investigated the effect of surface mass transfer on hypersonic viscous interactions . 
    46	0.14	what are the existing solutions for hypersonic viscous interactions over an insulated flat plate . 
    70	0.14	experimental results on hypersonic viscous interaction . 


We see that in some places texts are leaking, which are combined with queries by insignificant terms, but at the same time the Jaccard coefficient - our ranking function! - is high.

# VSM

Let's try to do the same now, but with tf-idf and cosine distance. We will do everything again "by hand", but "in real life" it is better to use efficient implementations of cosine distance, for example, from the scipy library.

In [15]:
from  sklearn.feature_extraction.text import TfidfVectorizer

tfidf_encoder = TfidfVectorizer()
tfidf_encoded_data = tfidf_encoder.fit_transform(query_data)
tfidf_encoded_queries = tfidf_encoder.transform(QUERIES)

list(tfidf_encoder.vocabulary_)[:3]

['what', 'similarity', 'laws']

## Task 2
Implement cosine distance

In [13]:
import numpy as np 

def cosine_distance(vector_a: np.array, vector_b: np.array) -> float:
  """
    Cosine distance: one minus dot product ratio
    to the product of L2 norms
  """
  return  1 - sum(vector_a * vector_b) / (np.linalg.norm(vector_a) * np.linalg.norm(vector_b))

# Simple text
assert cosine_distance(np.array([1, 0, 1, 1, 1]), np.array([0, 0, 1, 0, 0])) == 0.5

Now we calculate the cosine distance closest between the vector representations of documents and queries

In [16]:
for q_id, query in enumerate(tfidf_encoded_queries):
  # cast to the desired type
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in tfidf_encoded_data]
  # cosine distance
  id2doc2similarity = [(doc_id, doc, cosine_distance(query, doc)) \
                       for doc_id, doc in enumerate(docs)]
  # sort
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=False)
  
  print("Q: %s\nFOUND:" % QUERIES[q_id])
  
  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))

Q: viscous interactions
FOUND:
    44	0.50	has anyone investigated the effect of surface mass transfer on hypersonic viscous interactions . 
    46	0.56	what are the existing solutions for hypersonic viscous interactions over an insulated flat plate . 
    71	0.64	what has been done about viscous interactions in relatively low reynolds number flows,  particularly at high mach numbers . 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  
